In [1]:

%load_ext autoreload
%autoreload 2
from loaders.loaders import *
#from dataset_dict import dataset_dict, output_path
#from loaders.RecordDistributor import RecordDistributor
#from loaders.CleanedEnglishDataset import WikiBookEn
from loaders.OtherDatasets import *
#from loaders.PilePythonDataset import PilePythonDataset

/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from loaders.RandomEnglishDataset import RandomEnglishDataset
#ds= load_dataset("suolyer/pile_stackexchange",streaming=True,split="validation")
#ds = load_dataset("Muennighoff/flan",streaming=True,split="train")
#ds=RandomEnglishDataset()

ds = load_dataset("kanhatakeyama/TanukiCorpus",streaming=True,split="train")

In [7]:
loader=(iter(ds))

In [27]:
for i in range(1000):
    print(next(loader))

{'text': 'カーマスリブの祝福のパワーを感じられる2ヶ月間だった・・・かなりプレイ時間多い+カーマスリブの祝福を1個取引所購入して、行動力溜まったら木こりを繰り返しても2ヶ月ぐらいかかった。そして普通に船カッコイイ。\n図面以外の材料が全部揃ってからスタートして、ハイデル20人ぐらい+カルフェオン10人ぐらい+エフェリア5人の大体35人ぐらいを造船所で働かせてたもんで、図面以外の作業が全行程終わるまで1週間ぐらいで終わっちゃって図面集めるだけの作業が10日以上続いた。もっとはやめに図面デイリーしとけばよかった・・・'}
{'text': 'Membranということで、どんなもんか疑問に思いながら聴いてみたのだが、少なくともこの1枚目に関しては正しくタイトル通り軽快なブギ・ウギのオンパレードで良い。1曲目のジョニー・オーティスからしてもうノリノリである。途中ライトニン・ホプキンスが入ってきたりして、それもブギ・ナンバー(Herald時代だろうか)なのだった。'}
{'text': 'ありがちなタイトルだが少しまとめてみます。案件でAppEngineを使って3ヶ月程度になりますが、限られた用途においてベストパフォーマンスを発揮することは間違いないと思います。しかしながらいくらかの不安点があるのも事実です。自分の思考を整理するためにもまとめてみます。\n全く何も考えずにスケールするとまでは行きませんが、インフラに対して考慮すべき諸々から解放されます。datastoreもスケーラビリティに優れています。バックエンドでCloud SQLを使用した場合はある程度DBがボトルネックになります。\nこれは説明するまでもないですがスクリプト言語に比べ型がある分、同じ処理をする場合に堅牢でしょう。またコンパイラが厳密すぎるほど厳密なので、不必要なimportでもエラーを出します。go fmtで標準でコードフォーマットする機能もあります。\nなんだかんだこれが大きいのですが、Goは体感として2010年以降に流行り始めた言語として最も勢いのある言語の一つだと思います。ほかはSwiftとかですかね。トレンドであることは情報が多いとか枯れる速度が早いという実利もあるけど、人が集まってるほうが賑やかで楽しいと思います。\ngoroutine便利らしいですね。ただ、appengineだとあ

In [22]:

loader=iter(ds)

In [7]:
import json
out_path="../data/eng/eng.jsonl"
out_path="/data/hatakeyama/python/eng_corpus/eng.jsonl"
from tqdm import tqdm
for i in tqdm(range(10**5)):
    t=next(loader)["text"]
    if t=="":
        continue

    with open(out_path,"a") as f:
        f.write(json.dumps({"text":t},ensure_ascii=False)+"\n")

100%|██████████| 100000/100000 [01:04<00:00, 1547.57it/s]
